In [137]:
from transformers import BertTokenizer, BertModel
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch import nn
import os
from dotenv import load_dotenv
from datasets import Dataset
from imblearn.over_sampling import RandomOverSampler  # For advanced oversampling


In [76]:
from pathlib import Path
from dotenv import load_dotenv
env_path = Path('/Users/pranavturlapati/COMP-560-Deepseekers/.env')
load_dotenv(dotenv_path=env_path)


True

In [77]:
text_df = pd.read_csv('/Users/pranavturlapati/COMP-560-Deepseekers/Data/Classification Data/inputs_text.csv')         # column: raw_text
json_df = pd.read_csv('/Users/pranavturlapati/COMP-560-Deepseekers/Data/Classification Data/inputs_json.csv') # column: json_output
label_df = pd.read_csv('/Users/pranavturlapati/COMP-560-Deepseekers/Data/Classification Data/outputs.csv')      # binary labels (0 or 1)

In [78]:
text_df.head()

,bkn_nop,sac_cha,cle_det,den_chi,nyk_cha,lal_okc,lac_sas
0,alvarado fires away and buries the three,hurry seth Hurry nobody stopped the ball oh th...,this is rush it up Malik be a step up two won'...,Bulls beat the Nuggets earlier in the season a...,was oh my goodness going to give him some vert...,set to go here at center court And away we go ...,great to see coach Van Dundy back as the ball ...
1,immediately after that miss not too often that...,damonte Sabonis has led the NBA in rebounding ...,Mobley on the perimeter flips to Donovan Mitch...,Murray's going to be guarded by giddy he's got...,hornets going to take on Thunder tomorrow in OKC,we have an incredible color scheme tonight uni...,it'll be Clipper ball this has truly felt like...
2,have you never experienced it yourself i I hav...,look at that fancy right-handed finger roll mi...,jayen Z scream for Dennis Sher all the way ins...,okay in the lane the shot is good Jordan who h...,first bucket for the Knicks carl Anthony Towns,We got the Lakers with their gold unis,zoo gets the first touch for the Clippers goin...
3,olen had it slapped away by Williams perfectly...,simpson takes his time and scores his first two,M with a high screen for Darius Garland gets i...,Murray is inside I was going to say a career h...,if you set the pace move around they can't use...,starts off things with a scoring of the two,he's made his last 12 tonight is Zoo's 77th ga...
4,alvarado buries the three jose Alvarado's seco...,nice little move in the paint there by Nerkage,easy down deep for a dunk to Jaylen Duran,vich over to Giddy buellis lets it go that's good,can you steal points like that good jeff with ...,Reeves had a clean look and the Lakers get a t...,dunn lobs to Derek Jones Jr look out below too...


In [79]:
json_df.head()

,bkn_nop,sac_cha,cle_det,den_chi,nyk_cha,lal_okc,lac_sas
0,"""{\""team\"":\""BKN\"",\""opponent\"":\""NOP\"",\""grap...","""{\""team\"":\""SAC\"",\""opponent\"":\""CHA\"",\""grap...","""{\""team\"":\""CLE\"",\""opponent\"":\""DET\"",\""grap...","""{\""team\"":\""DEN\"",\""opponent\"":\""CHI\"",\""grap...","""{\""team\"":\""NYK\"",\""opponent\"":\""CHA\"",\""grap...","{\team\"":\""LAL\"",\""opponent\"":\""OKC\"",\""graph\...","{\team\"":\""LAC\"",\""opponent\"":\""SAS\"",\""graph\..."
1,"""{\""team\"":\""BKN\"",\""opponent\"":\""NOP\"",\""grap...","""{\""team\"":\""SAC\"",\""opponent\"":\""CHA\"",\""grap...","""{\""team\"":\""CLE\"",\""opponent\"":\""DET\"",\""grap...","""{\""team\"":\""DEN\"",\""opponent\"":\""CHI\"",\""grap...","""{\""team\"":\""NYK\"",\""opponent\"":\""CHA\"",\""grap...","{\team\"":\""LAL\"",\""opponent\"":\""OKC\"",\""graph\...","{\team\"":\""LAC\"",\""opponent\"":\""SAS\"",\""graph\..."
2,"""{\""team\"":\""BKN\"",\""opponent\"":\""NOP\"",\""grap...","""{\""team\"":\""SAC\"",\""opponent\"":\""CHA\"",\""grap...","""{\""team\"":\""CLE\"",\""opponent\"":\""DET\"",\""grap...","""{\""team\"":\""DEN\"",\""opponent\"":\""CHI\"",\""grap...","""{\""team\"":\""NYK\"",\""opponent\"":\""CHA\"",\""grap...","{\team\"":\""LAL\"",\""opponent\"":\""OKC\"",\""graph\...","{\team\"":\""LAC\"",\""opponent\"":\""SAS\"",\""graph\..."
3,"""{\""team\"":\""BKN\"",\""opponent\"":\""NOP\"",\""grap...","""{\""team\"":\""SAC\"",\""opponent\"":\""CHA\"",\""grap...","""{\""team\"":\""CLE\"",\""opponent\"":\""DET\"",\""grap...","""{\""team\"":\""DEN\"",\""opponent\"":\""CHI\"",\""grap...","""{\""team\"":\""NYK\"",\""opponent\"":\""CHA\"",\""grap...","{\team\"":\""LAL\"",\""opponent\"":\""OKC\"",\""graph\...","{\team\"":\""LAC\"",\""opponent\"":\""SAS\"",\""graph\..."
4,"""{\""team\"":\""BKN\"",\""opponent\"":\""NOP\"",\""grap...","""{\""team\"":\""SAC\"",\""opponent\"":\""CHA\"",\""grap...","""{\""team\"":\""CLE\"",\""opponent\"":\""DET\"",\""grap...","""{\""team\"":\""DEN\"",\""opponent\"":\""CHI\"",\""grap...","""{\""team\"":\""NYK\"",\""opponent\"":\""CHA\"",\""grap...","{\team\"":\""LAL\"",\""opponent\"":\""OKC\"",\""graph\...","{\team\"":\""LAC\"",\""opponent\"":\""SAS\"",\""graph\..."


In [80]:
label_df.head()

,bkn_nop,sac_cha,cle_det,den_chi,nyk_cha,lal_okc,lac_sas
0,1,1,1,1,1,1,1
1,1,1,1,1,0,1,1
2,1,1,1,1,1,1,1
3,1,1,1,1,0,1,1
4,1,1,1,1,1,1,1


In [81]:
df_text_long = text_df.melt(var_name='game_id', value_name='text')
df_json_long = json_df.melt(var_name='game_id', value_name='json_output')
df_label_long = label_df.melt(var_name='game_id', value_name='label')


In [82]:
df_text_long['row'] = df_text_long.index
df_json_long['row'] = df_json_long.index
df_label_long['row'] = df_label_long.index

In [83]:
merged = df_text_long.merge(df_json_long, on=['game_id', 'row']).merge(df_label_long, on=['game_id', 'row'])


In [84]:
merged = merged[['game_id', 'text', 'json_output', 'label']]

In [85]:
merged.head()

,game_id,text,json_output,label
0,bkn_nop,alvarado fires away and buries the three,"""{\""team\"":\""BKN\"",\""opponent\"":\""NOP\"",\""grap...",1
1,bkn_nop,immediately after that miss not too often that...,"""{\""team\"":\""BKN\"",\""opponent\"":\""NOP\"",\""grap...",1
2,bkn_nop,have you never experienced it yourself i I hav...,"""{\""team\"":\""BKN\"",\""opponent\"":\""NOP\"",\""grap...",1
3,bkn_nop,olen had it slapped away by Williams perfectly...,"""{\""team\"":\""BKN\"",\""opponent\"":\""NOP\"",\""grap...",1
4,bkn_nop,alvarado buries the three jose Alvarado's seco...,"""{\""team\"":\""BKN\"",\""opponent\"":\""NOP\"",\""grap...",1


In [86]:
print(merged.head().to_markdown(index=False))  # For a nice Markdown-style table

| game_id   | text                                                                                                                                                   | json_output                                                                                                                                               |   label |
|:----------|:-------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------|--------:|
| bkn_nop   | alvarado fires away and buries the three                                                                                                               | "{\"team\":\"BKN\",\"opponent\":\"NOP\",\"graph\":true,\"player_graph\":true,\"players\":[\"Alvarado\"],\"player_stat\":\"3P%\",\"team_stat\":null}"      |       1 |
|

In [87]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Combine the text and JSON as input to BERT
merged['input'] = merged['text'] + ' [SEP] ' + merged['json_output']

# Tokenize
tokens = tokenizer(
    list(merged['input']),
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors='np'  # Use numpy arrays instead
)




In [88]:
dataset = Dataset.from_dict({
    'input_ids': tokens['input_ids'],
    'attention_mask': tokens['attention_mask'],
    'label': merged['label'].values
})

In [141]:
def oversample_dataset(dataset):
    class_0 = dataset.filter(lambda x: x['label'] == 0)
    return concatenate_datasets([dataset, class_0, class_0])  # 3x oversampling

AttributeError: 'DatasetDict' object has no attribute 'train_test_split'

In [ ]:
labels = merged['label']  # Assuming train_dataset is already loaded
class_counts = np.bincount(labels)
pos_weight = len(labels) / (2 * class_counts[1])  # Weight for class 1
neg_weight = len(labels) / (2 * class_counts[0])  # Weight for class 0
class_weights = torch.tensor([neg_weight, pos_weight], dtype=torch.float32)

In [133]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from transformers import EarlyStoppingCallback

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    
    # Calculate for both classes (0 and 1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average=None, labels=[0, 1], zero_division=0
    )
    
    return {
        "precision_0": precision[0],  # Minority class
        "recall_0": recall[0],
        "f1_0": f1[0],
        "precision_1": precision[1],   # Majority class
        "recall_1": recall[1],
        "f1_1": f1[1],
        "f1_macro": (f1[0] + f1[1]) / 2,  # Macro-average F1
    }

In [100]:
from transformers import BertForSequenceClassification, AutoModelForSequenceClassification, DistilBertForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

In [134]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",  
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_steps=10,           # Log metrics every 10 steps
    logging_first_step=True,    # Log the first step
    no_cuda=True,              # Disable CUDA if no GPU
    load_best_model_at_end=True,  
    metric_for_best_model="f1_0",  # Track minority class F1
    greater_is_better=True,     # Higher F1 is better
    report_to="none",           # Disable external logging
    remove_unused_columns=True, # Saves memory
)

# Initialize Trainer
trainer = Trainer(
    model=model,  # Your pre-loaded model
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stop if no improvement
)


In [135]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision 0,Recall 0,F1 0,Precision 1,Recall 1,F1 1,F1 Macro
1,0.360600,0.395394,0.000000,0.000000,0.000000,0.876923,1.000000,0.934426,0.467213
2,0.342400,0.437598,0.000000,0.000000,0.000000,0.876923,1.000000,0.934426,0.467213
3,0.303400,0.483708,0.000000,0.000000,0.000000,0.876923,1.000000,0.934426,0.467213


TrainOutput(global_step=99, training_loss=0.3124171171826546, metrics={'train_runtime': 290.5391, 'train_samples_per_second': 3.538, 'train_steps_per_second': 0.454, 'total_flos': 78449233377240.0, 'train_loss': 0.3124171171826546, 'epoch': 3.0})

In [74]:
for lr in [1e-5, 2e-5, 3e-5, 5e-5]:
    print(f"\n🔁 Training with learning rate = {lr}")

    # Reload a fresh model each time to ensure fairness
    model = AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased", num_labels=2
    )

    training_args = TrainingArguments(
        output_dir=f"./results/lr_{lr}",     # Unique directory per run
        eval_strategy="epoch",
        save_strategy="no",
        learning_rate=lr,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        num_train_epochs=1,
        weight_decay=0.01,
        logging_steps=500,
        no_cuda=True,
        disable_tqdm=True,
        report_to="none",
        load_best_model_at_end=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
    )

    trainer.train()
    eval_result = trainer.evaluate()
    print(f"📉 Final eval loss for lr={lr}: {eval_result['eval_loss']:.4f}")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🔁 Training with learning rate = 1e-05
{'eval_loss': 0.6787074208259583, 'eval_runtime': 1.9287, 'eval_samples_per_second': 33.701, 'eval_steps_per_second': 2.592, 'epoch': 1.0}
{'train_runtime': 49.163, 'train_samples_per_second': 5.228, 'train_steps_per_second': 0.671, 'train_loss': 0.6993870590672349, 'epoch': 1.0}


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.6787074208259583, 'eval_runtime': 1.9357, 'eval_samples_per_second': 33.58, 'eval_steps_per_second': 2.583, 'epoch': 1.0}
📉 Final eval loss for lr=1e-05: 0.6787

🔁 Training with learning rate = 2e-05
{'eval_loss': 0.6737611293792725, 'eval_runtime': 1.8934, 'eval_samples_per_second': 34.33, 'eval_steps_per_second': 2.641, 'epoch': 1.0}
{'train_runtime': 45.4722, 'train_samples_per_second': 5.652, 'train_steps_per_second': 0.726, 'train_loss': 0.7003947749282374, 'epoch': 1.0}


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.6737611293792725, 'eval_runtime': 1.909, 'eval_samples_per_second': 34.05, 'eval_steps_per_second': 2.619, 'epoch': 1.0}
📉 Final eval loss for lr=2e-05: 0.6738

🔁 Training with learning rate = 3e-05
{'eval_loss': 0.6806286573410034, 'eval_runtime': 2.1168, 'eval_samples_per_second': 30.707, 'eval_steps_per_second': 2.362, 'epoch': 1.0}
{'train_runtime': 46.7305, 'train_samples_per_second': 5.5, 'train_steps_per_second': 0.706, 'train_loss': 0.7069802717729048, 'epoch': 1.0}


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 0.6806286573410034, 'eval_runtime': 1.9302, 'eval_samples_per_second': 33.675, 'eval_steps_per_second': 2.59, 'epoch': 1.0}
📉 Final eval loss for lr=3e-05: 0.6806

🔁 Training with learning rate = 5e-05
{'eval_loss': 0.667644739151001, 'eval_runtime': 1.8807, 'eval_samples_per_second': 34.561, 'eval_steps_per_second': 2.659, 'epoch': 1.0}
{'train_runtime': 48.901, 'train_samples_per_second': 5.256, 'train_steps_per_second': 0.675, 'train_loss': 0.7169795758796461, 'epoch': 1.0}
{'eval_loss': 0.667644739151001, 'eval_runtime': 1.8983, 'eval_samples_per_second': 34.242, 'eval_steps_per_second': 2.634, 'epoch': 1.0}
📉 Final eval loss for lr=5e-05: 0.6676


In [59]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.6514582633972168, 'eval_runtime': 4.5306, 'eval_samples_per_second': 14.347, 'eval_steps_per_second': 1.986, 'epoch': 3.0}


In [91]:
print(merged['label'].value_counts(normalize=True))

label
1    0.878882
0    0.121118
Name: proportion, dtype: float64


In [90]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'label'],
        num_rows: 257
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'label'],
        num_rows: 65
    })
})